In [1]:
!pip install session_info
!pip install pycaret

  Using cached session_info-1.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 KB 3.5 MB/s eta 0:00:00
  Using cached pycaret-3.0.2-py3-none-any.whl (483 kB)
  Using cached category_encoders-2.6.1-py2.py3-none-any.whl (81 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached lightgbm-3.3.5-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached schemdraw-0.15-py3-none-any.whl (106 kB)
  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)
  Using cached importlib_metadata-6.7.0-py3-none-any.whl (22 kB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
  Using cached plotly-5.15.0-py2.py3-none-any.whl (15.5 MB)
  Using cached yellowbrick-1.5-py3-none-any.whl (282 kB)
  Using cached sktime-0.17.0-py3-none-any.whl (16.1 MB)
  Using cached tbats-1.1.3-py3-none-any.whl (44 kB)
  Using cached pmdarima-2.0.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinu

#### Import librairies

In [2]:
import pandas as pd
import os
import re

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset , random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

import random

from pycaret.classification import *
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

import session_info

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
session_info.show()

#### Set the random seed for reproducibility

In [3]:
seed = 64
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#### Get the data

In [4]:
raw_data= pd.read_csv('data/train.csv', low_memory=False)
data_test= pd.read_csv('data/test.csv', low_memory=False)

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 365 entries, Patient_ID to Type_of_Venom_Allergy_IGE_Venom
dtypes: float64(322), int64(32), object(11)
memory usage: 8.3+ MB


#### Looking which are the targets to predict

In [6]:
missing_cols = set(raw_data.columns) ^ set(data_test.columns)
print(missing_cols)
len(missing_cols)

{'Type_of_Respiratory_Allergy_GINA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram', 'Type_of_Food_Allergy_Other', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Egg', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Food_Allergy', 'trustii_id', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Food_Allergy_Shellfish', 'Type_of_Food_Allergy_Aromatics', 'Type_of_Food_Allergy_Tree_Nuts', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Severe_Allergy', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Food_Allergy_Peanut', 'Type_of_Food_Allergy_Oral_Syndrom', 'Allergy_Present', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Other_Legumes', 'Respiratory_Allergy', 'Type_of_Respiratory_Allerg

30

## Data Pre-processing

### Preprocessing for the train set

In [7]:
liste_of_Targets =['Allergy_Present', 'Severe_Allergy', 'Respiratory_Allergy', 'Food_Allergy', 'Venom_Allergy',
                     'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_CONJ', 
                     'Type_of_Respiratory_Allergy_GINA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                     'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree',
                     'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach',
                     'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Aromatics', 'Type_of_Food_Allergy_Other',
                     'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish',
                     'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Food_Allergy_Mammalian_Milk', 
                     'Type_of_Food_Allergy_Oral_Syndrom', 'Type_of_Food_Allergy_Other_Legumes', 'Type_of_Food_Allergy_Peanut',
                     'Type_of_Food_Allergy_Shellfish', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Tree_Nuts',
                     'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Venom_Allergy_IGE_Venom']
def preprocessing_data(df):
    df = df.drop('Food_Type_0', axis =1)
    df.replace(-1, 0, inplace=True)
    data_noNAN = df.fillna(-1)
    # obtain Targets
    Targets = data_noNAN.loc[:,liste_of_Targets]
    # filter feautures
    X1=data_noNAN.loc[:, ['Chip_Type','Age','Gender','French_Residence_Department','Blood_Month_sample']]
    X= data_noNAN.iloc[:, 8:-29]
    data = pd.concat( [X1, X] , axis=1)
    # handle the 'Treatment_of_rhinitis' feature
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].astype(str)
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].str.replace('.0', '', regex=True)
    
    ##  Get_dummies of the 'object' type columns
    
    columns_to_encode = ['Chip_Type', 'French_Residence_Department', 'French_Region',
         'Treatment_of_athsma', 'Age_of_onsets',
       'General_cofactors', 'Treatment_of_atopic_dematitis','Treatment_of_rhinitis']
    
    ### Split the columns using multiple delimiters and create dummy columns
    dummy_dfs = []
    for col in columns_to_encode:
        # Split the data in the column that use  delimiters
        data[col] = data[col].astype(str)
        data[col] = data[col].apply(lambda x: [i.strip() for i in re.split('[,.]', x)])

        # Create dummy columns
        dummy_df = pd.get_dummies(data[col].apply(pd.Series).stack(), prefix=f"{col}", prefix_sep='_').groupby(level=0).sum()
        dummy_dfs.append(dummy_df)

    ### Concatenate the original DataFrame with the dummy columns
    df_final = pd.concat([data] + dummy_dfs, axis=1)

    ### Drop the original columns from the final dataset
    df_final.drop(columns=columns_to_encode, inplace=True)
    
    # Converting all values into 'float16' type
    encode_data = df_final.astype('float16')
    print(encode_data.info())
    
    return encode_data,Targets

In [8]:
encode_data,Targets = preprocessing_data(raw_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 467 entries, Age to Treatment_of_rhinitis_9
dtypes: float16(467)
memory usage: 2.7 MB
None


### Preprocessing for the test set

In [9]:
def preprocessing_data_test(df):
    df = df.drop('Food_Type_0', axis =1)
    df.replace(-1, 0, inplace=True)
    data_test_noNAN = df.fillna(-1)
    # filter feautures
    X1=data_test_noNAN.loc[:, ['Chip_Type']]
    X= data_test_noNAN.iloc[:, 5:]
    data = pd.concat( [X1, X] , axis=1)
    # handle the 'Treatment_of_rhinitis' feature
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].astype(str)
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].str.replace('.0', '', regex=True)
    # handle 'Age_of_onsets' which don't have the same format in data test and train
    data['Age_of_onsets'] = data['Age_of_onsets'].astype(str)

    
    ##  Get_dummies of the 'object' type columns
    
    columns_to_encode = ['Chip_Type', 'French_Residence_Department', 'French_Region',
         'Treatment_of_athsma', 'Age_of_onsets',
       'General_cofactors', 'Treatment_of_atopic_dematitis','Treatment_of_rhinitis']
    
    ### Split the columns using multiple delimiters and create dummy columns
    dummy_dfs = []
    for col in columns_to_encode:
        # Split the data in the column that use  delimiters
        data[col] = data[col].astype(str)
        data[col] = data[col].apply(lambda x: [i.strip() for i in re.split('[,.]', x)])

        # Create dummy columns
        dummy_df = pd.get_dummies(data[col].apply(pd.Series).stack(), prefix=f"{col}", prefix_sep='_').groupby(level=0).sum()
        dummy_dfs.append(dummy_df)

    ### Concatenate the original DataFrame with the dummy columns
    df_final = pd.concat([data] + dummy_dfs, axis=1)

    ### Drop the original columns from the final dataset
    df_final.drop(columns=columns_to_encode, inplace=True)
    
    # Converting all values into 'float16' type
    encode_data = df_final.astype('float16')
    print(encode_data.info())
    
    return encode_data


In [10]:
encode_data_test = preprocessing_data_test(data_test)
missing_cols = set(encode_data.columns) ^ set(encode_data_test.columns)
print(missing_cols)
len(missing_cols)
encode_data_test = encode_data_test.reindex(columns=encode_data.columns, fill_value=0).astype('float16')
encode_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Columns: 444 entries, Age to Treatment_of_rhinitis_9
dtypes: float16(444)
memory usage: 508.3 KB
None
{'Treatment_of_athsma_8', 'Treatment_of_atopic_dematitis_7', 'French_Residence_Department_deptJJJ', 'French_Residence_Department_deptOOO', 'French_Residence_Department_deptIII', 'French_Residence_Department_deptTTT', 'French_Residence_Department_deptRRR', 'French_Residence_Department_deptU', 'French_Residence_Department_deptCCCC', 'French_Region_regionO', 'French_Residence_Department_deptHHH', 'French_Residence_Department_deptNNN', 'French_Residence_Department_deptDD', 'French_Residence_Department_deptQQQ', 'French_Residence_Department_deptPPP', 'French_Residence_Department_deptK', 'French_Residence_Department_deptUU', 'French_Residence_Department_deptW', 'General_cofactors_11', 'French_Residence_Department_deptMMM', 'French_Residence_Department_deptDDD', 'French_Residence_Department_deptAAAA', 'French_Region_regio

#### Pycaret part

In [11]:
import os

def obtain_all_methods_imbl (encode_data, Y, list_of_metods):
    
    liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']
    for method in list_of_metods:
        print(method)
        
        folder_path = f'Test_Imbl_{method}_tuned'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        for column in Y.columns:
            if column not in liste_column_monovalue :
                rows_with_9 = Y[column].isin([9])
                Targets_without_9 = Y[column][~rows_with_9].reset_index(drop=True)
                X = encode_data[~rows_with_9].reset_index(drop=True)

                dataset= pd.concat([X,Targets_without_9], axis = 1)
                model = setup(data= dataset, target= column, train_size=0.85, fix_imbalance=True, fix_imbalance_method=method)
                best = compare_models(sort = 'F1', include = ['rf', 'et', 'lightgbm','ada'])
                tuned_best= tune_model(best,n_iter= 50, optimize = 'F1',fold = 10)
                file_name = f"{column}_ETC_brutforce_imb_binary"
                file_path = os.path.join(folder_path, file_name)
                save_model(tuned_best, file_path)
            else:
                continue

In [ ]:
List_methods_imbl=['RandomOverSampler','SMOTE','BorderlineSMOTE']  
obtain_all_methods_imbl (encode_data, Targets, List_methods_imbl) 

RandomOverSampler


,Description,Value
0,Session id,7951
1,Target,Allergy_Present
2,Target type,Binary
3,Original data shape,"(1759, 468)"
4,Transformed data shape,"(2590, 468)"
5,Transformed train set shape,"(2326, 468)"
6,Transformed test set shape,"(264, 468)"
7,Numeric features,467
8,Preprocess,1
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8474,0.8875,0.9372,0.8759,0.9052,0.5151,0.5263,0.4540
rf,Random Forest Classifier,0.8421,0.8779,0.9140,0.8871,0.9001,0.5228,0.5260,0.7950
lightgbm,Light Gradient Boosting Machine,0.8354,0.8870,0.9053,0.8859,0.8953,0.5096,0.5119,0.6010
ada,Ada Boost Classifier,0.7886,0.8584,0.7893,0.9282,0.8528,0.4860,0.5068,0.4330


Processing:   0%|          | 0/21 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7800,0.5000,1.0000,0.7800,0.8764,0.0000,0.0000
1,0.7800,0.5000,1.0000,0.7800,0.8764,0.0000,0.0000
2,0.7800,0.5000,1.0000,0.7800,0.8764,0.0000,0.0000
3,0.7733,0.5000,1.0000,0.7733,0.8722,0.0000,0.0000
4,0.7733,0.5000,1.0000,0.7733,0.8722,0.0000,0.0000
5,0.7785,0.5000,1.0000,0.7785,0.8755,0.0000,0.0000
6,0.7785,0.5000,1.0000,0.7785,0.8755,0.0000,0.0000
7,0.7785,0.5000,1.0000,0.7785,0.8755,0.0000,0.0000
8,0.7785,0.5000,1.0000,0.7785,0.8755,0.0000,0.0000


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9846,0.8668,0.1333,0.2333,0.1567,0.1537,0.1665,3.8810
ada,Ada Boost Classifier,0.9802,0.5890,0.0833,0.1500,0.1067,0.0988,0.1035,3.3280
rf,Random Forest Classifier,0.9846,0.8807,0.0000,0.0000,0.0000,0.0000,0.0000,3.0230
et,Extra Trees Classifier,0.9839,0.8853,0.0000,0.0000,0.0000,-0.0010,-0.0011,2.9780


Processing:   0%|          | 0/21 [00:00<?, ?it/s]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:54:23
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 50 candidates, totalling 500 fits


### Generate predictions from classifiers and include them into the dataset + evaluation 

In [55]:
data= pd.read_csv('data/train.csv', low_memory=False)
data_test= pd.read_csv('data/test.csv', low_memory=False)

### Define which target will be predict at the end with which model

In [56]:
list_of_Target_predict_with_pytorch_meta = ['Type_of_Food_Allergy_Other_Legumes', 'Type_of_Food_Allergy_Mammalian_Milk', 
                  'Venom_Allergy', 'Type_of_Food_Allergy_Tree_Nuts', 'Type_of_Food_Allergy_Aromatics', 
                  'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom',
                  'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut', 
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_TPO', 
                  'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Food_Allergy_Shellfish']

In [89]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.features = self.data.iloc[:, :-1].values
        self.labels = self.data.iloc[:, -1].values

        if self.transform is not None:
            self.features, self.labels = self.transform.fit_resample(self.features, self.labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(int(self.labels[idx]), dtype=torch.long)

        return features, label
    
    @property
    def num_features(self):
        return self.features.shape[1]
    
    @property
    def num_classes(self):
        return len(set(self.labels))

    
class Allergy_Net(nn.Module):
    
    def __init__(self,input_size,hidden_size, num_class,dropout_rate):

        super(Allergy_Net,self).__init__()
        #self.subsampler = RandomFeatureSubsampler(p=0.2)
        self.linear1= nn.Linear(input_size,hidden_size)
        self.linear2= nn.Linear(hidden_size,int(hidden_size/8))
        self.linear3= nn.Linear(int(hidden_size/8),int(hidden_size/32))
        self.linear4= nn.Linear(int(hidden_size/32),num_class)

        self.dropout1 = nn.Dropout(dropout_rate)
        self.batchnorm1 = nn.BatchNorm1d(int(hidden_size/32))

    def forward(self,inputs):
        #inputs = self.subsampler(inputs)
        x = torch.relu(self.linear1(inputs))
        x= self.dropout1(x)
        x = torch.relu(self.linear2(x))
        x= self.dropout1(x)
        x = torch.relu(self.linear3(x))
        x= self.dropout1(x)
        x= self.batchnorm1(x)
        outputs= self.linear4(x)

        # no softmax because Cross entropy Loss
        return outputs
    


### Obtain pred with pycaret Smote only

In [58]:
def obtain_pred(data, folder_path):
    liste=[]
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            print("Calling model:", file) 
            model = load_model(os.path.join(subdir, os.path.splitext(file)[0]))
            values=predict_model(model, data= data.reset_index())
            file_name = os.path.splitext(file)[0]
            values_name = '{}_{}'.format(file_name, 'values')
            globals()[values_name] = values
            globals()[values_name] = globals()[values_name].rename(columns={'prediction_label': 'pred_label ' + str(values_name), 'prediction_score': 'pred_score ' + str(values_name) })
            liste.append(globals()[values_name])
        return liste

In [74]:
folder_path_pycaret = 'Test_Imbl_SMOTE_tuned'
pred_pycaret= obtain_pred(encode_data.reset_index(drop=True), folder_path_pycaret)

Calling model: Type_of_Food_Allergy_Aromatics_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Respiratory_Allergy_CONJ_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Food_Allergy_Peanut_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Tree_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Venom_Allergy_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Food_Allergy_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Food_Allergy_Fruits_and_Vegetables_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Food_Allergy_Shellfish_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model

In [73]:
liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df = pd.DataFrame()
for i,df in enumerate(pred_pycaret):
    if i==0:
        merged_df = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df = pd.concat([merged_df, last_two_columns], axis=1)
        
added_list = ['pred_label ' + s + '_ETC_brutforce_imb_binary_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df)), columns= added_list)
merged_df = pd.concat([merged_df, zeros_data], axis=1)
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 524 entries, index to pred_label Type_of_Food_Allergy_Cereals_&_Seeds_ETC_brutforce_imb_binary_values
dtypes: float32(467), float64(27), int32(1), int64(29)
memory usage: 6.6 MB


,index,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,Act_d_5,...,pred_label Type_of_Food_Allergy_Fish_ETC_brutforce_imb_binary_values,pred_score Type_of_Food_Allergy_Fish_ETC_brutforce_imb_binary_values,pred_label Type_of_Venom_Allergy_ATCD_Venom_ETC_brutforce_imb_binary_values,pred_score Type_of_Venom_Allergy_ATCD_Venom_ETC_brutforce_imb_binary_values,pred_label Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_ETC_brutforce_imb_binary_values,pred_score Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_ETC_brutforce_imb_binary_values,pred_label Type_of_Respiratory_Allergy_IGE_Pollen_Herb_ETC_brutforce_imb_binary_values,pred_score Type_of_Respiratory_Allergy_IGE_Pollen_Herb_ETC_brutforce_imb_binary_values,pred_label Type_of_Food_Allergy_Other_ETC_brutforce_imb_binary_values,pred_label Type_of_Food_Allergy_Cereals_&_Seeds_ETC_brutforce_imb_binary_values
0,0,15.0,0.0,7.0,1.0,1.0,0.0,0.0,0.000000,0.0,...,0,0.6424,0,0.5383,0,0.9994,0,0.9961,0,0
1,1,72.0,1.0,5.0,9.0,1.0,9.0,0.0,0.000000,0.0,...,0,0.6098,0,0.5492,0,0.9995,0,0.9961,0,0
2,2,67.0,1.0,6.0,9.0,0.0,9.0,0.0,0.000000,0.0,...,0,0.6098,0,0.5624,0,0.9998,0,0.9975,0,0
3,3,13.0,1.0,9.0,1.0,1.0,9.0,0.0,2.150391,0.0,...,0,0.7264,0,0.7138,1,0.9970,1,0.9558,0,0
4,4,28.0,1.0,12.0,9.0,1.0,1.0,0.0,74.000000,0.0,...,0,0.6646,0,0.6503,1,0.9993,1,0.5495,0,0


### Obtain pred by pytorch_fold_meta

In [62]:
folder_path_fold = 'liste_classif_optim_2_no_oversampled'
folder_path_meta='liste_meta_4'
liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

def get_predictions_fold_imblearn_meta_classifier(df,list_of_targets, folder_path_fold,folder_path_meta):
    
    liste=[]
    for column in list_of_targets.columns:
        print(column)
        dataset_panda= pd.concat([df,list_of_targets[column]], axis = 1).reset_index(drop=True)
        dataset_all=CustomDataset(dataset_panda)
        train_all = DataLoader(dataset_all, batch_size=64, shuffle=False)

       
        if column not in liste_column_monovalue:
            input_size= 467 
            hidden_size= 1024
            num_class = 2 
            dropout_rate=0.3
            
            model_list = []
            target_string = f"{column}_fold"

            # Iterate over the files in the folder
            for filename in os.listdir(folder_path_fold):
                # Check if the file path contains the target string
                if target_string in filename:
                    # Load the model using the appropriate method
                    model = Allergy_Net(input_size,hidden_size,num_class,dropout_rate)
                    model_path = os.path.join(folder_path_fold, filename)
                    model.load_state_dict(torch.load(model_path))
                    model_list.append(model)

            # obtain prediction for each folds
            predictions_all = []
            for model in model_list:
                model.eval()
                pred_model=[]
                with torch.no_grad():
                    for data, labels in train_all:
                        outputs = model(data)
                        pred_model.extend(outputs)
                predictions_all.append(pred_model)

            # Combine predictions from different folds
            combined_pred = []
            for i in range(len(predictions_all[0])):
                combined_pred_mini = []
                for j in range(len(predictions_all)):
                    combined_pred_mini.extend(predictions_all[j][i])
                combined_pred.append(combined_pred_mini)
                
            # Extract values from tensors and convert to a list of lists
            combined_pred_values = [[value.item() for value in sample] for sample in combined_pred]
            num_folds = len(predictions_all)*2
            # Create a dictionary to store the fold predictions
            fold_predictions = {}
            for i in range(num_folds):
                fold_predictions[f"Value_{i+1}"] = [sample[i].item() for sample in combined_pred]

            # Convert the dictionary into a DataFrame
            df_combined_pred = pd.DataFrame(fold_predictions)


            target_string_for_meta_imbl = f"{column}_meta_model_pycaret_binary"
            for filename in os.listdir(folder_path_meta):
                # Check if the file path contains the target string
                if target_string_for_meta_imbl in filename:
                    model = load_model(os.path.join(folder_path_meta, os.path.splitext(filename)[0]))
                    values=predict_model(model, data= df_combined_pred.reset_index())
                    file_name = os.path.splitext(filename)[0]
                    values_name = '{}_{}'.format(file_name, 'values')
                    globals()[values_name] = values
                    globals()[values_name] = globals()[values_name].rename(columns={'prediction_label': 'pred_label ' + str(values_name), 'prediction_score': 'pred_score ' + str(values_name) })
                    liste.append(globals()[values_name])
                    print('done')
    return liste

In [63]:
pred_pytorch_meta=get_predictions_fold_imblearn_meta_classifier(encode_data,Targets, folder_path_fold,folder_path_meta)

Allergy_Present
Transformation Pipeline and Model Successfully Loaded
done
Severe_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Respiratory_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Food_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Venom_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_ARIA
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_CONJ
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_GINA
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Pollen_Gram
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Pollen_Herb
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Pollen_Tree
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Dander_Ani

In [64]:
print(len(pred_pytorch_meta))
pred_pytorch_meta[0]

27


,index,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,Value_8,Value_9,Value_10,Value_11,Value_12,Value_13,Value_14,Value_15,Value_16,pred_label Allergy_Present_meta_model_pycaret_binary_values,pred_score Allergy_Present_meta_model_pycaret_binary_values
0,0,-0.748127,0.729992,-0.584334,0.499938,-0.773140,0.906865,-0.365902,0.172279,-0.376860,0.111737,-0.633374,0.795414,-0.585548,0.792772,-0.906501,0.757275,1,1.00
1,1,0.746651,-0.758935,0.341197,-0.434825,0.950985,-0.915116,0.418233,-0.683620,0.324738,-0.618463,0.632095,-0.547063,0.668416,-0.474834,0.404382,-0.541455,0,0.83
2,2,0.733035,-0.744944,0.363207,-0.454483,0.988670,-0.956325,0.420199,-0.676870,0.331931,-0.624628,0.657758,-0.573595,0.695211,-0.501030,0.441308,-0.576205,1,1.00
3,3,-0.814914,0.790831,-1.250587,1.151908,-1.247099,1.370371,-1.376382,1.003550,-0.441291,0.180355,-1.376143,1.567106,-1.131734,1.325826,-1.204989,1.048573,1,1.00
4,4,2.561285,-2.592341,1.212610,-1.193888,2.119431,-2.285433,-1.933193,1.469598,0.540509,-0.800075,1.962318,-2.170416,2.523063,-2.252595,3.708384,-3.600559,0,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,2984,-0.786242,0.773249,-0.847874,0.780075,-0.214157,0.318150,-0.090434,-0.136489,-0.115012,-0.155921,-0.068002,0.183874,-0.054706,0.246645,-0.621908,0.472636,1,0.74
2985,2985,0.634945,-0.641316,0.682803,-0.726152,1.071488,-1.048074,0.201980,-0.465010,0.309130,-0.606897,1.063268,-1.031496,0.911311,-0.705643,0.790858,-0.903036,0,0.88
2986,2986,-1.022694,1.008510,-1.007091,0.946322,-0.863912,1.000217,-0.347636,0.157731,-0.616937,0.385259,-0.822320,1.007491,-0.686823,0.898095,-0.907244,0.757713,1,1.00
2987,2987,-1.474948,1.441920,-0.980786,0.898808,-1.064659,1.189412,-0.841738,0.569061,-0.989572,0.851761,-1.010729,1.221723,-0.905806,1.116985,-1.044051,0.890789,1,1.00


In [65]:
liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df_pytorch = pd.DataFrame()
for i,df in enumerate(pred_pytorch_meta):
    if i==0:
        merged_df_pytorch = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df_pytorch = pd.concat([merged_df_pytorch, last_two_columns], axis=1)
        
added_list = ['pred_label ' + s + '_meta_model_pycaret_binary_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df_pytorch)), columns= added_list)
merged_df_pytorch = pd.concat([merged_df_pytorch, zeros_data], axis=1)
merged_df_pytorch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Data columns (total 73 columns):
 #   Column                                                                                      Non-Null Count  Dtype  
---  ------                                                                                      --------------  -----  
 0   index                                                                                       2989 non-null   int32  
 1   Value_1                                                                                     2989 non-null   float32
 2   Value_2                                                                                     2989 non-null   float32
 3   Value_3                                                                                     2989 non-null   float32
 4   Value_4                                                                                     2989 non-null   float32
 5   Value_5                                  

In [80]:
df_predictions_final= pd.DataFrame()

for target in Targets:
    if target not in list_of_Target_predict_with_pytorch_meta:
        column_target= merged_df[f'pred_label {target}_ETC_brutforce_imb_binary_values']
        df_predictions_final = pd.concat([df_predictions_final, column_target], axis=1)
        
    else:
        column_target= merged_df_pytorch[f'pred_label {target}_meta_model_pycaret_binary_values']
        df_predictions_final = pd.concat([df_predictions_final, column_target], axis=1) 
df_predictions_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2989 entries, 0 to 2988
Data columns (total 29 columns):
 #   Column                                                                                     Non-Null Count  Dtype
---  ------                                                                                     --------------  -----
 0   pred_label Allergy_Present_ETC_brutforce_imb_binary_values                                 2989 non-null   int64
 1   pred_label Severe_Allergy_ETC_brutforce_imb_binary_values                                  2989 non-null   int64
 2   pred_label Respiratory_Allergy_ETC_brutforce_imb_binary_values                             2989 non-null   int64
 3   pred_label Food_Allergy_ETC_brutforce_imb_binary_values                                    2989 non-null   int64
 4   pred_label Venom_Allergy_meta_model_pycaret_binary_values                                  2989 non-null   int64
 5   pred_label Type_of_Respiratory_Allergy_ARIA_ETC_brutforce_imb_binar

#### homogenize columns names for evaluation

In [81]:
for target in Targets:
    liste_string_to_check = [f' {target}_m',f' {target}_E']
    for elem in liste_string_to_check:
        for column in df_predictions_final.columns:
            if elem in column:
                new_column_name = column.replace(column, f'pred_label_{target}_values')
                df_predictions_final = df_predictions_final.rename(columns={column: new_column_name})
df_predictions_final.info()
df_predictions_final.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2989 entries, 0 to 2988
Data columns (total 29 columns):
 #   Column                                                            Non-Null Count  Dtype
---  ------                                                            --------------  -----
 0   pred_label_Allergy_Present_values                                 2989 non-null   int64
 1   pred_label_Severe_Allergy_values                                  2989 non-null   int64
 2   pred_label_Respiratory_Allergy_values                             2989 non-null   int64
 3   pred_label_Food_Allergy_values                                    2989 non-null   int64
 4   pred_label_Venom_Allergy_values                                   2989 non-null   int64
 5   pred_label_Type_of_Respiratory_Allergy_ARIA_values                2989 non-null   int64
 6   pred_label_Type_of_Respiratory_Allergy_CONJ_values                2989 non-null   int64
 7   pred_label_Type_of_Respiratory_Allergy_GINA_values      

,pred_label_Allergy_Present_values,pred_label_Severe_Allergy_values,pred_label_Respiratory_Allergy_values,pred_label_Food_Allergy_values,pred_label_Venom_Allergy_values,pred_label_Type_of_Respiratory_Allergy_ARIA_values,pred_label_Type_of_Respiratory_Allergy_CONJ_values,pred_label_Type_of_Respiratory_Allergy_GINA_values,pred_label_Type_of_Respiratory_Allergy_IGE_Pollen_Gram_values,pred_label_Type_of_Respiratory_Allergy_IGE_Pollen_Herb_values,...,pred_label_Type_of_Food_Allergy_Fruits_and_Vegetables_values,pred_label_Type_of_Food_Allergy_Mammalian_Milk_values,pred_label_Type_of_Food_Allergy_Oral_Syndrom_values,pred_label_Type_of_Food_Allergy_Other_Legumes_values,pred_label_Type_of_Food_Allergy_Peanut_values,pred_label_Type_of_Food_Allergy_Shellfish_values,pred_label_Type_of_Food_Allergy_TPO_values,pred_label_Type_of_Food_Allergy_Tree_Nuts_values,pred_label_Type_of_Venom_Allergy_ATCD_Venom_values,pred_label_Type_of_Venom_Allergy_IGE_Venom_values
0,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,1,1,1,1,...,0,0,1,0,0,0,0,0,0,0


#### Evaluation binary for each methods

In [82]:
data_true= pd.read_csv('data/train.csv', low_memory=False)
data_true.head()

,Patient_ID,Chip_Code,Chip_Type,Chip_Image_Name,Age,Gender,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,...,Type_of_Food_Allergy_Fruits_and_Vegetables,Type_of_Food_Allergy_Mammalian_Milk,Type_of_Food_Allergy_Oral_Syndrom,Type_of_Food_Allergy_Other_Legumes,Type_of_Food_Allergy_Peanut,Type_of_Food_Allergy_Shellfish,Type_of_Food_Allergy_TPO,Type_of_Food_Allergy_Tree_Nuts,Type_of_Venom_Allergy_ATCD_Venom,Type_of_Venom_Allergy_IGE_Venom
0,PCR0232,02AHX0C9,ALEX,02AHX0C9.bmp,15.0,0.0,7.0,deptGGG,regionF,1,...,0,0,0,0,0,0,0,0,0,0
1,QVW0271,AD53727_2,ISAC_V1,NaN,72.0,1.0,5.0,deptY,regionD,9,...,9,9,9,9,9,9,9,9,0,0
2,QVW0489,AM91127_4,ISAC_V1,NaN,67.0,1.0,6.0,deptY,regionD,9,...,0,0,1,0,0,0,0,0,0,0
3,TXV0075,881309006950,ISAC_V1,AC10727_4_881309006950_2013_10_28_21_20_5.bmp,13.0,1.0,9.0,deptEE,regionC,1,...,0,0,0,0,0,0,0,0,0,0
4,QVW0946,CSB5927_4,ISAC_V1,NaN,28.0,1.0,12.0,deptY,regionD,9,...,9,9,9,9,9,9,9,9,0,0


In [83]:
target_columns = ['Type_of_Food_Allergy_Other_Legumes','Type_of_Food_Allergy_Cereals_&_Seeds', 'Allergy_Present', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_Other',
                  'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Food_Allergy', 'Type_of_Food_Allergy_Oral_Syndrom','Type_of_Food_Allergy_Tree_Nuts', 'Severe_Allergy',
                  'Type_of_Food_Allergy_Aromatics', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut',
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_GINA', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast',
                  'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Food_Allergy_Shellfish']
f1_scores=[]
below_threshold_columns=[]
threshold =0.89
for column in target_columns:
    # Get the predicted and true labels for the column
    
    rows_with_9 = data_true[column].isin([9])
    Targets_without_9 = data_true[column][~rows_with_9]
    merged_df_removal = df_predictions_final[~rows_with_9]
        
    merged_df_mod = pd.concat([merged_df_removal, Targets_without_9] , axis=1).reset_index(drop=True)
    y_true = merged_df_mod[column]
    y_pred = merged_df_mod[f'pred_label_{column}_values']
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate recall
    recall = recall_score(y_true, y_pred)
    
    # Calculate F1 score
    if (data_true[column] == 1).any():
        f1 = f1_score(y_true, y_pred,average='macro')
    else:
        f1 = f1_score(y_true, y_pred, pos_label=0)
    cm = confusion_matrix(y_true, y_pred)

    # Display the metrics and confusion matrix
    print(f"Metrics for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print("Confusion Matrix:")
    print(cm)
    print()
    if f1 != 0:
        f1_scores.append(f1) # Append the F1 score to the list
    if f1 < threshold:
        below_threshold_columns.append(column)

mean_f1 = sum(f1_scores) / len(f1_scores)  # Calculate the mean of F1 scores

print(f"Mean F1 Score: {mean_f1}")
print(below_threshold_columns)

Metrics for Type_of_Food_Allergy_Other_Legumes:
Accuracy: 0.9931549471064095
Recall: 1.0
F1 Score: 0.9486779722035925
Confusion Matrix:
[[1546   11]
 [   0   50]]

Metrics for Type_of_Food_Allergy_Cereals_&_Seeds:
Accuracy: 1.0
Recall: 0.0
F1 Score: 1.0
Confusion Matrix:
[[1607]]

Metrics for Allergy_Present:
Accuracy: 0.9766913018760659
Recall: 0.9890350877192983
F1 Score: 0.9659484677778423
Confusion Matrix:
[[ 365   26]
 [  15 1353]]

Metrics for Type_of_Food_Allergy_Mammalian_Milk:
Accuracy: 0.9981331673926571
Recall: 1.0
F1 Score: 0.9712235798320332
Confusion Matrix:
[[1579    3]
 [   0   25]]

Metrics for Type_of_Food_Allergy_Other:
Accuracy: 1.0
Recall: 0.0
F1 Score: 1.0
Confusion Matrix:
[[1607]]

Metrics for Type_of_Respiratory_Allergy_IGE_Mite_Cockroach:
Accuracy: 0.9900464499004645
Recall: 0.9882352941176471
F1 Score: 0.9899495141487098
Confusion Matrix:
[[820   7]
 [  8 672]]

Metrics for Venom_Allergy:
Accuracy: 0.9959852793576447
Recall: 1.0
F1 Score: 0.9220599739243807
C

## Generate prediction for the test set

In [84]:
original_validation_data = pd.read_csv('data/test.csv')

data_test = original_validation_data.set_index('trustii_id')

In [85]:
encode_data_test = preprocessing_data_test(data_test)
missing_cols = set(encode_data.columns) ^ set(encode_data_test.columns)
print(missing_cols)
len(missing_cols)
encode_data_test = encode_data_test.reindex(columns=encode_data.columns, fill_value=0).astype('float16')
encode_data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586 entries, 1 to 1282
Columns: 443 entries, Gender to Treatment_of_rhinitis_9
dtypes: float16(443)
memory usage: 527.8 KB
None
{'Treatment_of_athsma_8', 'Treatment_of_atopic_dematitis_7', 'French_Residence_Department_deptJJJ', 'French_Residence_Department_deptOOO', 'French_Residence_Department_deptIII', 'French_Residence_Department_deptTTT', 'French_Residence_Department_deptRRR', 'French_Residence_Department_deptU', 'Age', 'French_Residence_Department_deptCCCC', 'French_Region_regionO', 'French_Residence_Department_deptHHH', 'French_Residence_Department_deptNNN', 'French_Residence_Department_deptDD', 'French_Residence_Department_deptQQQ', 'French_Residence_Department_deptPPP', 'French_Residence_Department_deptK', 'French_Residence_Department_deptUU', 'French_Residence_Department_deptW', 'General_cofactors_11', 'French_Residence_Department_deptMMM', 'French_Residence_Department_deptDDD', 'French_Residence_Department_deptAAAA', 'French_R

In [91]:
pred_pycaret= obtain_pred(encode_data_test.reset_index(), folder_path_pycaret)

liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df = pd.DataFrame()
for i,df in enumerate(pred_pycaret):
    if i==0:
        merged_df = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df = pd.concat([merged_df, last_two_columns], axis=1)
        
added_list = ['pred_label ' + s + '_ETC_brutforce_imb_binary_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df)), columns= added_list)
merged_df = pd.concat([merged_df, zeros_data], axis=1)
merged_df.info()

Calling model: Type_of_Food_Allergy_Aromatics_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Respiratory_Allergy_CONJ_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Food_Allergy_Peanut_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Tree_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Venom_Allergy_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Food_Allergy_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Food_Allergy_Fruits_and_Vegetables_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model Successfully Loaded
Calling model: Type_of_Food_Allergy_Shellfish_ETC_brutforce_imb_binary.pkl
Transformation Pipeline and Model

In [92]:
merged_df.head()

,index,trustii_id,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,...,pred_label Type_of_Food_Allergy_Fish_ETC_brutforce_imb_binary_values,pred_score Type_of_Food_Allergy_Fish_ETC_brutforce_imb_binary_values,pred_label Type_of_Venom_Allergy_ATCD_Venom_ETC_brutforce_imb_binary_values,pred_score Type_of_Venom_Allergy_ATCD_Venom_ETC_brutforce_imb_binary_values,pred_label Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_ETC_brutforce_imb_binary_values,pred_score Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_ETC_brutforce_imb_binary_values,pred_label Type_of_Respiratory_Allergy_IGE_Pollen_Herb_ETC_brutforce_imb_binary_values,pred_score Type_of_Respiratory_Allergy_IGE_Pollen_Herb_ETC_brutforce_imb_binary_values,pred_label Type_of_Food_Allergy_Other_ETC_brutforce_imb_binary_values,pred_label Type_of_Food_Allergy_Cereals_&_Seeds_ETC_brutforce_imb_binary_values
0,0,1,0.0,1.0,6.0,9.0,1.0,0.0,0.0,0.000000,...,0,0.5733,0,0.6803,1,0.9995,0,0.9820,0,0
1,1,4,0.0,1.0,7.0,9.0,1.0,1.0,0.0,0.109985,...,0,0.5295,0,0.5489,1,0.9950,0,0.9933,0,0
2,2,5,0.0,0.0,10.0,1.0,1.0,1.0,0.0,0.000000,...,0,0.5427,0,0.6187,0,0.9095,1,0.9884,0,0
3,3,7,0.0,1.0,8.0,0.0,1.0,0.0,0.0,0.000000,...,0,0.6487,0,0.5608,0,0.9567,0,0.9948,0,0
4,4,8,0.0,0.0,4.0,1.0,0.0,9.0,0.0,0.000000,...,0,0.5544,0,0.5868,0,0.9998,0,0.9986,0,0


In [94]:
num_rows = len(encode_data_test)  # Specify the number of rows
data_array = np.random.choice([0, 1], size=(num_rows, 29))

# Create a DataFrame using the numpy array
Targets_test = pd.DataFrame(data_array, columns=['Allergy_Present', 'Severe_Allergy', 'Respiratory_Allergy', 'Food_Allergy', 'Venom_Allergy',
                     'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_CONJ', 
                     'Type_of_Respiratory_Allergy_GINA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                     'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree',
                     'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach',
                     'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Aromatics', 'Type_of_Food_Allergy_Other',
                     'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish',
                     'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Food_Allergy_Mammalian_Milk', 
                     'Type_of_Food_Allergy_Oral_Syndrom', 'Type_of_Food_Allergy_Other_Legumes', 'Type_of_Food_Allergy_Peanut',
                     'Type_of_Food_Allergy_Shellfish', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Tree_Nuts',
                     'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Venom_Allergy_IGE_Venom'])

Targets_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 29 columns):
 #   Column                                          Non-Null Count  Dtype
---  ------                                          --------------  -----
 0   Allergy_Present                                 586 non-null    int64
 1   Severe_Allergy                                  586 non-null    int64
 2   Respiratory_Allergy                             586 non-null    int64
 3   Food_Allergy                                    586 non-null    int64
 4   Venom_Allergy                                   586 non-null    int64
 5   Type_of_Respiratory_Allergy_ARIA                586 non-null    int64
 6   Type_of_Respiratory_Allergy_CONJ                586 non-null    int64
 7   Type_of_Respiratory_Allergy_GINA                586 non-null    int64
 8   Type_of_Respiratory_Allergy_IGE_Pollen_Gram     586 non-null    int64
 9   Type_of_Respiratory_Allergy_IGE_Pollen_Herb     586 non-null    i

In [96]:
pred_pytorch_meta=get_predictions_fold_imblearn_meta_classifier(encode_data_test.reset_index(drop=True),Targets_test, folder_path_fold,folder_path_meta)

Allergy_Present
Transformation Pipeline and Model Successfully Loaded
done
Severe_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Respiratory_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Food_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Venom_Allergy
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_ARIA
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_CONJ
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_GINA
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Pollen_Gram
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Pollen_Herb
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Pollen_Tree
Transformation Pipeline and Model Successfully Loaded
done
Type_of_Respiratory_Allergy_IGE_Dander_Ani

In [97]:
merged_df_pytorch = pd.DataFrame()
for i,df in enumerate(pred_pytorch_meta):
    if i==0:
        merged_df_pytorch = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df_pytorch = pd.concat([merged_df_pytorch, last_two_columns], axis=1)
        
added_list = ['pred_label ' + s + '_meta_model_pycaret_binary_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df_pytorch)), columns= added_list)
merged_df_pytorch = pd.concat([merged_df_pytorch, zeros_data], axis=1)
merged_df_pytorch.info()

df_predictions_final= pd.DataFrame()

for target in Targets:
    if target not in list_of_Target_predict_with_pytorch_meta:
        column_target= merged_df[f'pred_label {target}_ETC_brutforce_imb_binary_values']
        df_predictions_final = pd.concat([df_predictions_final, column_target], axis=1)
        
    else:
        column_target= merged_df_pytorch[f'pred_label {target}_meta_model_pycaret_binary_values']
        df_predictions_final = pd.concat([df_predictions_final, column_target], axis=1) 
df_predictions_final.info()


for target in Targets:
    liste_string_to_check = [f' {target}_m',f' {target}_E']
    for elem in liste_string_to_check:
        for column in df_predictions_final.columns:
            if elem in column:
                new_column_name = column.replace(column, f'pred_label_{target}_values')
                df_predictions_final = df_predictions_final.rename(columns={column: new_column_name})
df_predictions_final.info()
df_predictions_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 73 columns):
 #   Column                                                                                      Non-Null Count  Dtype  
---  ------                                                                                      --------------  -----  
 0   index                                                                                       586 non-null    int32  
 1   Value_1                                                                                     586 non-null    float32
 2   Value_2                                                                                     586 non-null    float32
 3   Value_3                                                                                     586 non-null    float32
 4   Value_4                                                                                     586 non-null    float32
 5   Value_5                                    

,pred_label_Allergy_Present_values,pred_label_Severe_Allergy_values,pred_label_Respiratory_Allergy_values,pred_label_Food_Allergy_values,pred_label_Venom_Allergy_values,pred_label_Type_of_Respiratory_Allergy_ARIA_values,pred_label_Type_of_Respiratory_Allergy_CONJ_values,pred_label_Type_of_Respiratory_Allergy_GINA_values,pred_label_Type_of_Respiratory_Allergy_IGE_Pollen_Gram_values,pred_label_Type_of_Respiratory_Allergy_IGE_Pollen_Herb_values,...,pred_label_Type_of_Food_Allergy_Fruits_and_Vegetables_values,pred_label_Type_of_Food_Allergy_Mammalian_Milk_values,pred_label_Type_of_Food_Allergy_Oral_Syndrom_values,pred_label_Type_of_Food_Allergy_Other_Legumes_values,pred_label_Type_of_Food_Allergy_Peanut_values,pred_label_Type_of_Food_Allergy_Shellfish_values,pred_label_Type_of_Food_Allergy_TPO_values,pred_label_Type_of_Food_Allergy_Tree_Nuts_values,pred_label_Type_of_Venom_Allergy_ATCD_Venom_values,pred_label_Type_of_Venom_Allergy_IGE_Venom_values
0,1,0,1,0,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,1,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,1,1,1,0,1,0,1,1,1,...,1,1,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
target_columns = ['Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Other_Legumes', 'Allergy_Present', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_Other',
                  'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Food_Allergy', 'Type_of_Food_Allergy_Oral_Syndrom','Type_of_Food_Allergy_Tree_Nuts', 'Severe_Allergy',
                  'Type_of_Food_Allergy_Aromatics', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut',
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_GINA', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast',
                  'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Food_Allergy_Shellfish']

for elem in target_columns:
    original_validation_data[elem]=  df_predictions_final[f'pred_label_{elem}_values']

In [100]:
original_validation_data.head(30)

,trustii_id,Patient_ID,Chip_Code,Chip_Type,Chip_Image_Name,Age,Gender,Blood_Month_sample,French_Residence_Department,French_Region,...,Type_of_Food_Allergy_Egg,Type_of_Food_Allergy_Fish,Type_of_Respiratory_Allergy_GINA,Respiratory_Allergy,Type_of_Food_Allergy_TPO,Type_of_Respiratory_Allergy_IGE_Pollen_Tree,Type_of_Food_Allergy_Fruits_and_Vegetables,Type_of_Respiratory_Allergy_IGE_Molds_Yeast,Type_of_Respiratory_Allergy_IGE_Dander_Animals,Type_of_Food_Allergy_Shellfish
0,1,PMP0156,22 262C 3858,ISAC_V2,NaN,8.0,1.0,6.0,deptBBB,regionJ,...,0,0,1,1,0,1,0,1,1,0
1,4,PCR0234,02AHX0DC,ALEX,02AHX0DC.bmp,14.0,1.0,7.0,deptL,regionD,...,0,0,1,1,0,0,0,0,1,0
2,5,PCR0532,02AUN372,ALEX,02AUN372.png,32.0,0.0,10.0,deptUUU,regionF,...,0,0,1,1,1,1,1,1,1,0
3,7,GJH0147,EKF3830_4,ISAC_V2,EKF3830_4_2200444337_2023_2_17_11_58_24.bmp,65.0,1.0,8.0,deptQ,regionF,...,1,0,0,0,0,0,0,0,0,0
4,8,TXV0009,881204001164,ISAC_V1,1G20027_2_881204001164_2012_4_25_18_32_58.bmp,5.0,0.0,4.0,deptII,regionC,...,0,0,0,0,0,0,0,0,0,0
5,9,PCR0118,02AFA752,ALEX,NaN,49.0,0.0,1.0,deptXXX,regionI,...,0,0,0,1,0,0,0,0,0,0
6,10,QVW0214,AB02627_3,ISAC_V1,NaN,6.0,1.0,2.0,deptY,regionD,...,0,0,1,1,0,1,0,1,1,0
7,15,TXV0157,881602013302,ISAC_V1,BAF4027_4_881602013302_2016_2_23_16_38_11.bmp,13.0,1.0,2.0,deptRR,regionB,...,0,0,1,1,0,0,0,0,0,0
8,18,WQW0190,223112546,ISAC_V2,END0E30_1_223112546_2023_1_3_16_20_19.bmp,12.0,0.0,11.0,deptOO,regionL,...,1,0,0,0,0,0,0,0,0,0
9,23,TXV0282,881903001372,ISAC_V1,CXG1527_3_881903001372_2019_3_14_3_51_59.bmp,8.0,0.0,3.0,deptEE,regionC,...,0,0,1,1,0,0,0,0,0,0


In [102]:
original_validation_data.to_csv('Submission_mix_pycaret_SMOTE_Pytorch_resampling_KStrat_fold_meta_imblearn.csv', index=False, encoding='UTF-8')

#### Logics is the same than the first submission but an fix_imbalanced_method is set to give models more samples from the minority class and less from the majority class.
#### one notebook for the three best sampling methods previously test to compare the results when the models are tuned